# Pandas - Verarbeitung, Analyse und Darstellung von Daten 
https://pandas.pydata.org/docs/

Datensatz der ZAMG - Zentralanstalt für Meteorologie und Geodynamik
https://data.hub.zamg.ac.at/

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
%matplotlib inline

Daten lesen

In [ ]:
pfad = "ZAMG_GRAZ_Datensatz_20000101_20221207.csv"
df=pd.read_csv(pfad, parse_dates=['time'])
df=df.set_index('time')
df

Daten bereinigen

In [ ]:
df.dropna(axis=1, how='any')

Daten speichern

In [ ]:
df.to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt") 

In [ ]:
df_dymola = df
df_dymola["unix_timestamp"] = df.index
df_dymola.unix_timestamp = df_dymola.unix_timestamp.apply(lambda x: x.timestamp())
df_dymola.set_index("unix_timestamp")

In [ ]:
cols = ["unix_timestamp", "TTX", "RSX", "LT2", "D6X"]

In [ ]:
df[cols].to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt", sep=" ")


Was bedeuten die Kürzel? - Metadaten

In [ ]:
pfad_metadaten = "ZAMG_Parameter-Metadaten.csv"
df_meta=pd.read_csv(pfad_metadaten, quotechar = '*')
df_meta

In [ ]:
pd.set_option('display.max_colwidth', None)                        #Zeigt gesamte Spalte

In [ ]:
df_meta[df_meta["Identifier"]=="D6X"]

In [ ]:
df_meta[df_meta["Identifier"].isin(df.columns)]

Datenanalyse mit Pandas

In [ ]:
df.describe()

Wann gab es Niederschlag über 30mm pro Stunde?

In [ ]:
df[df['RSX']>30]

In [ ]:
len(df[df['RSX']<0.0000001])/34216

Visualisieren

In [ ]:
df['LT2'].hist()

In [ ]:
df['TTX'].plot()

In [ ]:
df['TTX']['2022'].plot()

Zeitreihen Resampling - Datengranularität

In [ ]:
df_TTX = df['TTX'].resample('ME').mean()

In [ ]:
df_TTX.plot()

In [ ]:
df_RSX = df['RSX'].resample('ME').sum()
df_RSX.plot()

Datenanalyse - Linearer Fit

In [ ]:
y=np.array(df['TTX'].dropna().values, dtype=float)
x=np.array(pd.to_datetime(df['TTX'].dropna()).index.values, dtype=float)
slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
xf = np.linspace(min(x),max(x),100)
xf1 = xf.copy()
xf1 = pd.to_datetime(xf1)
yf = (slope*xf)+intercept

In [ ]:
f, ax = plt.subplots(1, 1)
df['TTX'].dropna().plot(ax=ax,marker='.', ls='')
ax.plot(xf1, yf,label='Linear fit', lw=3)
plt.ylabel('Temperatur')
ax.legend();

Wie viel ist die mittlere Lufttemperatur in 2m Höhe vor der Uni Graz seit 2000 gestiegen?

In [ ]:
yf[-1]-yf[0]

Daten speichern

In [ ]:
df_dymola = df.loc["2020-12"]

In [ ]:
df_dymola.LT2.plot()

In [ ]:
df_dymola["LT2"].fillna(0).plot()

In [ ]:
df_dymola["LT2"].bfill().plot()

In [ ]:
df_dymola["LT2"].ffill().plot()

In [ ]:
df_dymola["LT2"].interpolate(method="polynomial", order=2).plot()

In [ ]:
df_dymola = df_dymola.interpolate(method="polynomial", order=2)

In [ ]:
df.to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt")

Datenformat UNIX Zeitstempel: Anzahl der seit dem 1. Januar 1970, 00:00:00 UTC, verstrichenen Sekunden 

In [ ]:
df_dymola = df_dymola.assign(unix_timestamp = df_dymola.index)
df_dymola.unix_timestamp = df_dymola.unix_timestamp.apply(lambda x: x.timestamp())
df_dymola = df_dymola.set_index("unix_timestamp")

In [ ]:
df_dymola.to_csv("ZAMG_GRAZ_Datensatz_20000101_20221207.txt", sep=" ")

In [ ]:
f = open("ZAMG_GRAZ_Datensatz_20000101_20221207.txt","r+")
lines = f.readlines() # lese File
f.seek(0) # zurück zum Anfang
f.write("#1\n") # Schreibe neue Zeile
f.write("double ZAMGdata"+str(df_dymola.shape)+" #") # Schreibe neue Zeile
for line in lines: # Schreibe ursprünglichen Inhalt
    f.write(line)
f.close()